# People Counter App with Openvino

### A people counter aplication optimized to work with low latency on the edge.
#### This model is optimized with the Openvino toolkit.

## Explaining Custom Layers

#### Let's understand how the openvino work flow is for converting a model.
#### First, the model is converted to an Intermediate Representative(IR) with the use of the model optimizer.
#### The Model Optimizer then searches the list of known layers for each layer contained in the input model topology before building the model's internal representation, optimizing the model, and producing the Intermediate Representation files.
#### The inference engine loads the IR format which is usually represented in '.xml' and '.bin' files. The inference engine loads the layers in a device plugin and the n search for all layers supported by the device.
#### If there's a layer not supported, the inference engine will report an error, and for such reasons it is advisable to go through the list of the layers and check if all are supported or not.

#### Note:  If a device doesn't support a particular layer, an alternative to creating a new custom layer is to target an additional device using the HETERO plugin. The Heterogeneous Plugin may be used to run an inference model on multiple devices allowing the unsupported layers on one device to "fallback" to run on another device (e.g., CPU) that does support those layers.

## Custom Layers Implementation

#### So yeah, there is a way to handle unsupported layers, one which is by simply running that unsupported layer in it's original framework, and another method is by adding custom layers.
#### But here we are concerned about custom layers...

### Model Optimizer

#### The Model Optimizer extracts information from the input model which includes the topology of the model layers along with parameters, input and output format, for each layer. The model is then optimized from the various known characteristics of the layers, then optimized model is output to IR format.
### There are 2 major custom layer extensions
#### *custom layer extractor*, which is reposible for identifying the custom layer operation and extracting the parameters for each instance of the custom layer.

#### *custom layer operation*, which is responsible for specifying the attributes that are supported by the custom layer and computing the output shape for each instance of the custom layer from its parameters

### Inference Engine

#### Each device plugin includes a library of optimized implementations to execute known layer operations which must be extended to execute a custom layer. The custom layer extension is implemented according to the target device:

### *Example of Custom Layer CPU Extension*
#### A compiled shared library (.so or .dll binary) needed by the CPU Plugin for executing the custom layer on the CPU.
### *Example of Custom Layer GPU Extension*
#### OpenCL source code (.cl) for the custom layer kernel that will be compiled to execute on the GPU along with a layer description file (.xml) needed by the GPU Plugin for the custom layer kernel.

### Using the model extension generator
#### The Model Extension Generator tool generates template source code files for each of the extensions needed by the Model Optimizer and the Inference Engine.

#### The script for this is available here- /opt/intel/openvino/deployment_tools/tools/extension_generator/extgen.py

*Arguments to configure extension generation in the interactive mode:*

##### --mo-op         generate a Model Optimizer operation

*complete script can be found at* https://docs.openvinotoolkit.org/2020.2/_docs_HOWTO_Custom_Layers_Guide.html 


## Why Should We Handle Custom Layers

#### In real world problems it's very important to be able to convert custom layers as team members might be developing something new and for your application to work smoothly you would probably need custom layers, let's say new piece of code was added to the model implementation, then a custom layer would be helpful to handle such layer.


## Model Research

#### I ended up using a model from Intel OpenVino Model Zoo due to poor performance of converted models. I stressed more  on model accuracy and inference time, I included model size as a another metric. I have stated the models I experimented with.

#### One of the models i experimented with is the ssd_mobilenet_v2_coco but it did not give very good accuracy, just like the other 2 tesnsorflow models i tried also

#### So the size of the models is of importance to us and we would like to optimize it as much as we can. for me to check the size of the files on the terminal i installed ncdu by running 
*sudo apt-get install ncdu*
#### Then to check for the sizes of files run, ncdu [filepath]

## *SSD_MOBILENET_v2_COCO*
### Before conversion

#### Size = 67 Mb
#### Inference Time = 50ms

### After conversion
#### Size = 65 Mb
#### Inference Time = 68ms

### Link to the model is http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

*To download to workspace run* 
wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

*To unpack the file in a linux envrironment run*

tar-xvf package-name.tar.gz


### I converted the model with the command below:
#### python /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config pipeline.config --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json --reverse_input_channel


### The main reason why i considered the ssd_mobilenet_v2_coco to not be a very good model for this task is because at some point in the video it stops detecting the same individual.

### I tried adjusting the output to make it's prediction better by reducing the confidence threshold as low as 0.4 but still the model did not give the right output.

#### The other 2 Models I used are ssd_inception_v2_coco *and* ssd_mobilenet_v1_coco

##### link to ssd_inception_v2_coco is http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz

##### link to ssd_mobilenet_v1_coco is http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz

## *SSD_INCEPTION_v2_COCO*
### Before conversion

#### Size = 98 MB
#### Inference Time = 150 ms

### After conversion
#### Size = 96 MB
#### Inference Time = 155ms

*To download to workspace run* 
wget http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz

*To unpack the file in a linux envrironment run*

tar-xvf package-name.tar.gz


### I converted the model with the command below:
#### python /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config pipeline.config --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json --reverse_input_channel


#### The ssd_inception_v2_coco model also failed to detect an individal that's been detected before and also the model file is quite large, so that's another downside in using this model for the task.

#### I also tried to imporve it's detection by reducing confidence threshold but no better result was gotten.

## *SSD_MOBILENET_v1_COCO*
### Before conversion

#### Size = 28 MB
#### Inference Time = 40 ms

### After conversion
#### Size = 26 MB
#### Inference Time = 44ms

*To download to workspace run* 
wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz

*To unpack the file in a linux envrironment run*

tar-xvf package-name.tar.gz


### I converted the model with the command below:
#### python /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model frozen_inference_graph.pb --tensorflow_object_detection_api_pipeline_config pipeline.config --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json --reverse_input_channel


#### The size of the model and the inference time of the model seems to be the best among the 3 models i have used but this model too fails to detect an individual it has detected before.
#### Tried again to adjust output by reducing confidence threshold but still no better result.

## The person-detection-retail-0013

#### The person-detection-retail-0013 seemed to be the killer model and even gives an inference time of about *44ms* and the size is mind-blowing compared to the converted models, just about *1.5MB*.

#### *To download the person-detection-retail-0013*
#### first run
*source /opt/intel/openvino/bin/setupvars.sh -pyver 3.5*

#### then
#### python3  /opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013 -o /home/workspace/model/pre_trained/intel


#### To run the app, use the code below

#### The Person-detection-retail model gives the best result with a confidence threshold of 0.6 and it detects all persons at all time and also considering it's small size, it is the best model for this task.

#### The output of the person-detection-retail-model on the ui gives a total count of peole in the video file and the average duration of a person spent in the video.

## Assess Model Use Cases

#### I beleive similar application could be widely useful, here are aspects where it can have impact.

### *Firstly, In a Numbering System*

#### I've been to churches and even gone for an open-ground concert and yeah i recall people still hand-counting. 

#### This is a very basic usecase but solves a problem
#### And in addition to numbering people it can also classifier the people based on categories, e.g height, gender, etc.. 



### *In the  Educational sysyem to control space management*

#### This application can be used in an examination environment.
#### *How the model should work* 
#### Firstly, the object dectection model would detect the total count of people, a bounding box will be drawn on all persons counted.
#### Then the distance between all bounding boxes should be calculated.
#### A confidence threshold is then set and if any any distance goes below that condidence threshold, then a photo is captured.

#### This usecase will help in curbing examination malpractises and will ease examination invigilators the stress of looking round the hall to avoid students from passing cheatsheets.

### Queing Systems

#### This one is quite a strategic one...
#### Yeah so i believe everyone gets annoyed probably when you go to a bank and then the queue is long, then you leave.
#### Banks can manage this by counting the amount of people on a queue using an object detection model
#### Then once the amount of people on that queue has reached a certain threshold, another queue is created.
#### This would help in customer retention

## Assess Effects on End User Needs

#### *Lighting, model accuracy, and camera focal length/image size have different effects on a deployed edge model. The potential effects of each of these are as follows:*


#### *Poor lighting* : Model's accuracy may fail dramatically or even completely drop close to zero. However, this can be avoided with good hardware that can process the images from poorly lit regions before passing it to the model.


#### *Decrease in model accuracy* : Decrease in model accuracy during conversion or other stages may make the model unusable if the model doesn't perform the required task such as detecting possibity of examination malpratice as mentioned above in use-cases. one solution to this would be to try other models as they will probably perform better .


#### *Distorted input from camera* : Distorted input from camera due to change in image size will affect the model in the sense that the model may not be able to properly process the input and then the distored input may not be detected properly by the model. An approach to solve this would be to use some augmnted images while training models and specifying the threshold skews.